In [ ]:
# medication specified with it's "code" i.e. zopiclone = "Q220426"
medication = "wd:" + "Q220426"

import sys
import json
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = f"""
    PREFIX bd: <http://www.bigdata.com/rdf#>
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT DISTINCT ?interactingDrug ?interactingDrugLabel ?sideEffect ?sideEffectLabel
    WHERE {{
        {medication} wdt:P769 ?interactingDrug. #P769 significant drug use.
        ?statement ps:P769 ?interactingDrug;   
                   pq:P1909 ?sideEffect.   #P1909 side effect

        SERVICE wikibase:label {{
            bd:serviceParam wikibase:language "en" . # get the lables in english
        }}
    }}
    LIMIT 100
"""

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Run the query
results = get_results(endpoint_url, query)

# Parse and store results in a list of dictionaries
parsed_results = []
for result in results["results"]["bindings"]:
    interacting_drug = result.get("interactingDrug", {}).get("value", "Unknown")
    interacting_drug_label = result.get("interactingDrugLabel", {}).get("value", "Unknown")
    side_effect = result.get("sideEffect", {}).get("value", "Unknown")
    side_effect_label = result.get("sideEffectLabel", {}).get("value", "No side effect listed")
    
    # Extract Q-code from the URI
    interacting_drug_code = interacting_drug.split("/")[-1] if interacting_drug != "Unknown" else "Unknown"
    side_effect_code = side_effect.split("/")[-1] if side_effect != "Unknown" else "Unknown"
    
    parsed_results.append({
        "interacting_drug": {
            "label": interacting_drug_label,
            "code": interacting_drug_code
        },
        "side_effect": {
            "label": side_effect_label,
            "code": side_effect_code
        }
    })

# Write results to a JSON file
output_file = "drug_interactions.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(parsed_results, f, indent=4, ensure_ascii=False)

print(f"Results successfully saved to {output_file}")

Results successfully saved to drug_interactions.json


In [17]:
# Input Medication A and B, zopiclone has negative sideffects when used together with pethidine.
medA = "wd:Q220426"  # Zopiclone
medB = "wd:Q55434"  # Pethidine

sleep_disorder = "wd:Q177190"

import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

endpoint_url = "https://query.wikidata.org/sparql"

def query_wikidata(query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Step 1: Check interaction
def check_interaction(medA, medB):
    query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT ?sideEffect ?sideEffectLabel
    WHERE {{
      {medA} wdt:P769 {medB}.
      ?statement ps:P769 {medB};
                 pq:P1909 ?sideEffect.

      SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "en".
      }}
    }}
    """
    return query_wikidata(query)

# Step 2: Find alternatives for Medication B
def find_alternatives(medB):
    query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT DISTINCT ?alternative ?alternativeLabel
    WHERE {{
      {medB} wdt:P2175 ?disorder.
      ?alternative wdt:P2175 ?disorder.
      FILTER(?alternative != {medB})
      SERVICE wikibase:label {{
        bd:serviceParam wikibase:language "en".
      }}
    }}
    """
    return query_wikidata(query)

# Step 3: Filter safe alternatives
def find_safe_alternatives(medA, alternatives):
    safe_alternatives = []
    for alt in alternatives:
        alt_code = alt["alternative"]["value"].split("/")[-1]
        query = f"""
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>

        ASK {{
          {medA} wdt:P769 wd:{alt_code}.
        }}
        """
        result = query_wikidata(query)
        if not result["boolean"]:
            safe_alternatives.append(alt)
    return safe_alternatives

# Step 3: Filter safe alternatives
def find_medicine_on_condition_treated(condition):

    query = f"""
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>

    SELECT DISTINCT ?medicine ?medicine_label
    WHERE {{
      ?medicine wdt:P2175 {condition}.
      ?medicine rdfs:label ?medicine_label
    FILTER (LANG(?medicine_label) = "en")
    }}
    """
    result = query_wikidata(query)
    print(result)
    formatted_result = []
    for item in result["results"]["bindings"]:
        formatted_result.append(f"{item["medicine_label"]["value"]}")
        
    print(formatted_result)
    return result

# Step 1: Check interaction
interaction_results = check_interaction(medA, medB)
print("Interaction Results:")
print(interaction_results)

# Step 2: Find alternatives for Medication B
alternatives_results = find_alternatives(medB)
alternatives = alternatives_results["results"]["bindings"]
print("Alternatives:")
print(alternatives)

# Step 3: Find safe alternatives
safe_alternatives = find_safe_alternatives(medA, alternatives)
print("Safe Alternatives:")
print(safe_alternatives)

find_medicine_on_condition_treated(sleep_disorder)

# Save results to CSV
df = pd.DataFrame(safe_alternatives)
df.to_csv("safe_alternatives.csv", index=False)
print("Safe alternatives saved to safe_alternatives.csv")


Interaction Results:
{'head': {'vars': ['sideEffect', 'sideEffectLabel']}, 'results': {'bindings': [{'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q123562764'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'depressed level of consciousness'}}, {'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q18557941'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'paralytic ileus'}}, {'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q123562764'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'depressed level of consciousness'}}, {'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3044895'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'central respiratory depression'}}, {'sideEffect': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q18557941'}, 'sideEffectLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'paralytic 

In [34]:
from SPARQLWrapper import SPARQLWrapper, JSON
endpoint_url = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(endpoint_url)
sparql.addCustomHttpHeader("User-Agent", "MySPARQLQuery/1.0 (example@example.com)")

In [29]:
# Gets the Drug Label
query = """
SELECT ?label
WHERE {
  wd:Q220426 rdfs:label ?label.
  FILTER (lang(?label) = "en")
}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results['results']['bindings'][0]['label']['value'])

zopiclone


In [30]:
# Get All Interacting Drugs wrt. another Drug (e.g., Zopiclone)
drug = "wd:Q220426"
interaction_property = "wdt:P769"
side_effect_property = "wdt:P1909"

query = f"""
SELECT ?interactingDrugLabel 
WHERE {{
  {drug} {interaction_property} ?interactingDrug.
  ?interactingDrug rdfs:label ?interactingDrugLabel.
  FILTER (LANG(?interactingDrugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for interacting_drug in results['results']['bindings']:
    print(interacting_drug['interactingDrugLabel']['value'])

codeine
pethidine
loxapine
morphine
alfentanil
(RS)-methadone
hydromorphone
dihydrocodeine
sodium oxybate
oxycodone
fentanyl
buprenorphine
tapentadol
sufentanil
periciazine
tramadol


In [35]:
drug = "wd:Q220426"  # Zopiclone
interaction_property = "p:P769"
interaction_statement = "ps:P769"
side_effect_qualifier = "pq:P1909"

query = f"""
SELECT ?interactingDrugLabel (GROUP_CONCAT(?sideEffectLabel; separator=", ") AS ?sideEffects) WHERE {{
  {drug} {interaction_property} ?statement.
  ?statement {interaction_statement} ?interactingDrug.
  ?interactingDrug rdfs:label ?interactingDrugLabel.
  FILTER (LANG(?interactingDrugLabel) = "en")

  OPTIONAL {{
    ?statement {side_effect_qualifier} ?sideEffect.
    ?sideEffect rdfs:label ?sideEffectLabel.
    FILTER (LANG(?sideEffectLabel) = "en")
  }}
}} GROUP BY ?interactingDrugLabel
"""

# Set query and format
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

# Execute query
results = sparql.query().convert()

# Prepare data for JSON
output_data = {}

for binding in results['results']['bindings']:
    interacting_drug_label = binding['interactingDrugLabel']['value']
    side_effects = binding.get('sideEffects', {}).get('value', 'No side effects listed')
    side_effects_list = side_effects.split(', ') if side_effects != 'No side effects listed' else []
    output_data[interacting_drug_label] = side_effects_list

# Write to JSON file
import json
output_file = "drug_side_effects.json"
with open(output_file, "w") as file:
    json.dump(output_data, file, indent=4)

print(f"Results successfully written to {output_file}")

Results successfully written to drug_side_effects.json


In [2]:
# Get all instances of "medicines"
has_use = "wdt:P366"
medication = "wd:Q12140"

query = f"""
SELECT ?drug ?drugLabel
WHERE {{
  ?drug {has_use} {medication}.  
  ?drug rdfs:label ?drugLabel.
  FILTER (LANG(?drugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
i=1
meds = []
for drug in results['results']['bindings']:
    print(str(i) + ":", drug['drugLabel']['value'])
    meds.append(drug['drugLabel']['value'])
    i += 1
  

import csv
output_file = 'medications.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    for med in meds:
        writer.writerow([med])

print(f"Medications have been written to {output_file}.")

1: ethanol
2: gabapentin
3: rac-salbutamol
4: isosorbide dinitrate
5: ampicillin
6: Diclofenac
7: D-sorbitol
8: DL-amphetamine
9: sodium bicarbonate
10: ciprofloxacin
11: α-streptozocin
12: L-menthol
13: carbon dioxide
14: carbamazepin
15: modafinil
16: iniparib
17: tioconazole
18: ibudilast
19: isoconazole
20: aprindine
21: nevirapine
22: sertraline
23: (+)-pseudoephedrine
24: ethoxzolamide
25: mitiglinide
26: cisatracurium besylate
27: phenprocoumon
28: benzylpenicillin
29: cilostazol
30: fusidic acid
31: natamycin
32: leflunomide
33: haloperidol
34: calcium carbonate
35: amber
36: aspirin
37: papaverine
38: phenytoin
39: phenobarbital
40: theophylline
41: fludarabine phosphate
42: (E)-phytonadione
43: dactinomycin
44: lysergic acid diethylamide
45: progesterone
46: apomorphine
47: capsaicin
48: aluminum hydroxide
49: chorionic gonadotropin
50: aspartame
51: arginine vasopressin
52: isoflurophate
53: 6-thioguanine
54: desoximetasone
55: ajmaline
56: natalizumab
57: ibuprofen
58: phys

In [ ]:
# Get the count of all medicines
has_use = "wdt:P366"
medication = "wd:Q12140"

query = f"""
SELECT (COUNT(?drug) AS ?drugCount)
WHERE {{
  ?drug {has_use} {medication}.  
  ?drug rdfs:label ?drugLabel.
  FILTER (LANG(?drugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print("Medicine Count:", results['results']['bindings'][0]['drugCount']['value'])


Medicine Count: 2689


In [ ]:
# Find Chemical Substance used as Medicine through Query by name
query = f"""
SELECT ?drug ?drugLabel
WHERE {{
  ?drug rdfs:label "zopiclone"@en.
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results['results']['bindings'][0]['drug']['value'])

http://www.wikidata.org/entity/Q220426


In [ ]:
# Get Best Matched Meds based on Wrongly Type Input Medication
from fuzzywuzzy import process

input_name = "zopliklon"  

matches = process.extract(input_name, meds, limit=5)
print(matches)


[('zopiclone', 78), ('eszopiclone', 70), ('xylometazoline', 60), ('tetrahydrozoline', 60), ('zotepine', 59)]


In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
endpoint_url = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(endpoint_url)

In [6]:
# Gets the Drug Label
query = """
SELECT ?label
WHERE {
  wd:Q220426 rdfs:label ?label.
  FILTER (lang(?label) = "en")
}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results['results']['bindings'][0]['label']['value'])

zopiclone


In [20]:
# Get All Interacting Drugs wrt. another Drug (e.g., Zopiclone)
drug = "wd:Q220426"
interaction_property = "wdt:P769"
side_effect_property = "wdt:P1909"

query = f"""
SELECT ?interactingDrugLabel 
WHERE {{
  {drug} {interaction_property} ?interactingDrug.
  ?interactingDrug rdfs:label ?interactingDrugLabel.
  FILTER (LANG(?interactingDrugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for interacting_drug in results['results']['bindings']:
    print(interacting_drug['interactingDrugLabel']['value'])

TypeError: unsupported operand type(s) for +: 'SPARQLWrapper' and 'str'

In [4]:
drug = "wd:Q220426"  # Zopiclone
interaction_property = "p:P769"
interaction_statement = "ps:P769"
side_effect_qualifier = "pq:P1909"

query = f"""
SELECT ?interactingDrugLabel (GROUP_CONCAT(?sideEffectLabel; separator=", ") AS ?sideEffects) WHERE {{
  {drug} {interaction_property} ?statement.
  ?statement {interaction_statement} ?interactingDrug.
  ?interactingDrug rdfs:label ?interactingDrugLabel.
  FILTER (LANG(?interactingDrugLabel) = "en")

  OPTIONAL {{
    ?statement {side_effect_qualifier} ?sideEffect.
    ?sideEffect rdfs:label ?sideEffectLabel.
    FILTER (LANG(?sideEffectLabel) = "en")
  }}
}} GROUP BY ?interactingDrugLabel
"""

# Set query and format
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

# Execute query
results = sparql.query().convert()

# Prepare data for JSON
output_data = {}

for binding in results['results']['bindings']:
    interacting_drug_label = binding['interactingDrugLabel']['value']
    side_effects = binding.get('sideEffects', {}).get('value', 'No side effects listed')
    side_effects_list = side_effects.split(', ') if side_effects != 'No side effects listed' else []
    output_data[interacting_drug_label] = side_effects_list

# Write to JSON file
import json
output_file = "drug_side_effects.json"
with open(output_file, "w") as file:
    json.dump(output_data, file, indent=4)

print(f"Results successfully written to {output_file}")

HTTPError: HTTP Error 403: Forbidden

In [ ]:
# Get all instances of "medicines"
has_use = "wdt:P366"
medication = "wd:Q12140"

query = f"""
SELECT ?drug ?drugLabel
WHERE {{
  ?drug {has_use} {medication}.  
  ?drug rdfs:label ?drugLabel.
  FILTER (LANG(?drugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
i=1
meds = []
for drug in results['results']['bindings']:
    print(str(i) + ":", drug['drugLabel']['value'])
    meds.append(drug['drugLabel']['value'])
    i += 1
  

import csv
output_file = 'medications.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    for med in meds:
        writer.writerow([med])

print(f"Medications have been written to {output_file}.")

1: carbon dioxide
2: DL-amphetamine
3: sodium bicarbonate
4: gabapentin
5: rac-salbutamol
6: ampicillin
7: Diclofenac
8: ethanol
9: isosorbide dinitrate
10: L-glutamine
11: aspartame
12: arginine vasopressin
13: lercanidipine
14: megestrol
15: neostigmine
16: boceprevir
17: phenobarbital
18: theophylline
19: magnesium hydroxide
20: tramadol
21: L-menthol
22: papaverine
23: phenytoin
24: oxycodone
25: fentanyl
26: cytarabine
27: tryptophan
28: buprenorphine
29: dextromethorphan
30: metformin
31: root vegetable
32: isoflurophate
33: 6-thioguanine
34: desoximetasone
35: ajmaline
36: natalizumab
37: physostigmine
38: butabarbital
39: meptazinol
40: ofatumumab
41: fluoxymesterone
42: sertraline
43: (+)-pseudoephedrine
44: ethoxzolamide
45: mitiglinide
46: cisatracurium besylate
47: phenprocoumon
48: benzylpenicillin
49: cilostazol
50: fusidic acid
51: iohexol
52: ambrisentan
53: heptabarbital
54: phenylacetic acid
55: D-sorbitol
56: fluorescein
57: insulin detemir
58: diphenylpyraline
59: f

In [ ]:
# Get the count of all medicines
has_use = "wdt:P366"
medication = "wd:Q12140"

query = f"""
SELECT (COUNT(?drug) AS ?drugCount)
WHERE {{
  ?drug {has_use} {medication}.  
  ?drug rdfs:label ?drugLabel.
  FILTER (LANG(?drugLabel) = "en")
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print("Medicine Count:", results['results']['bindings'][0]['drugCount']['value'])


Medicine Count: 2689


In [5]:
# Find Chemical Substance used as Medicine through Query by name
query = f"""
SELECT ?drug ?drugLabel
WHERE {{
  ?drug rdfs:label "zopiclone"@en.
}}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results['results']['bindings'][0]['drug']['value'])

http://www.wikidata.org/entity/Q220426


In [6]:
med_link = results['results']['bindings'][0]['drug']['value']
# extract the Q-code from the URI
med_code = med_link.split("/")[-1]



Q220426


In [ ]:
# Get Best Matched Meds based on Wrongly Type Input Medication
from fuzzywuzzy import process

input_name = "zopliklon"  

matches = process.extract(input_name, meds, limit=5)
print(matches)


[('zopiclone', 78), ('eszopiclone', 70), ('xylometazoline', 60), ('tetrahydrozoline', 60), ('zotepine', 59)]
